# Assignemt 2 - Section 2

### Part 1: PCA method with mean-filling 

In [2]:
import pandas as pd
ratings = pd.read_csv(r'dataset\ratings.csv')

In [3]:
ratings

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858
...,...,...,...,...
32000199,200948,79702,4.5,1294412589
32000200,200948,79796,1.0,1287216292
32000201,200948,80350,0.5,1294412671
32000202,200948,80463,3.5,1350423800


#### 1- calculate the average for each of the target items (I1, and I2)

In [4]:
target_items = [1, 2]

In [5]:
item_means = ratings[ratings['movieId'].isin(target_items)].groupby('movieId')['rating'].mean()
item_means

movieId
1    3.897438
2    3.275758
Name: rating, dtype: float64

In [ ]:
for item_id in target_items:
    if item_id in item_means.index:
        print(f"Mean rating for (I{target_items.index(item_id)+1}): {item_means[item_id]:.4f}")
    else:
        print(f"Item {item_id} not found in dataset.")

Mean rating for  (I1): 3.8974
Mean rating for  (I2): 3.2758


In [12]:
all_item_means = ratings.groupby('movieId')['rating'].mean()
print(f"\nTotal items with ratings: {len(all_item_means)}")


Total items with ratings: 84432


#### 2. Use the mean-filling method to replace the unspecified ratings of each of the target items (I1 and I2) with its corresponding mean value.

In [ ]:
# i will let this error just to let you know why we filtered our data later
user_item_matrix = ratings.pivot(index='userId', columns='movieId', values='rating')
user_item_matrix

C:\Users\HP\AppData\Local\Temp\ipykernel_28948\1902829208.py:1: PerformanceWarning: The following operation may generate 16966441536 cells in the resulting pandas object.
  user_item_matrix = ratings.pivot(index='userId', columns='movieId', values='rating')


MemoryError: Unable to allocate 15.8 GiB for an array with shape (16966441536,) and data type bool

In [ ]:
#Select Top 1,000 Popular Movies (plus target items if they aren't in top 1000)
desired_item_count = 1000

In [15]:
item_counts = ratings['movieId'].value_counts()
top_movies = item_counts.nlargest(desired_item_count).index.tolist()

In [16]:
for t in target_items:
    if t not in top_movies:
        top_movies.append(t)

ratings_filtered = ratings[ratings['movieId'].isin(top_movies)]

In [ ]:
#Select Top 100,000 Active Users
desired_user_count = 100000
user_counts = ratings_filtered['userId'].value_counts()
top_users = user_counts.nlargest(desired_user_count).index.tolist()

In [18]:
ratings_final = ratings_filtered[ratings_filtered['userId'].isin(top_users)]

In [19]:
ratings_final.shape

(15899442, 4)

In [20]:
ratings_final['userId'].nunique()

100000

In [21]:
ratings_final['movieId'].nunique()

1000

In [22]:
user_item_matrix = ratings_final.pivot(index='userId', columns='movieId', values='rating')
user_item_matrix

movieId,1,2,3,5,6,7,10,11,16,17,...,168252,171763,174055,176371,177765,187593,195159,202439,204698,207313
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,3.5,NaN,NaN,NaN,NaN,4.0,4.0,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,2.5,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,4.0,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,5.0,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200943,3.5,2.5,NaN,NaN,3.5,4.5,3.5,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200944,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
for item_id in target_items:
    if item_id in user_item_matrix.columns:
        # Recalculate mean on the filtered data to be accurate
        mean_val = user_item_matrix[item_id].mean() 
        
        user_item_matrix[item_id] = user_item_matrix[item_id].fillna(mean_val)
        print(f"Item {item_id}: Filled missing values with mean {mean_val:.4f}")
    else:
        print(f"Warning: Item {item_id} was lost during filtering (should not happen).")

Item 1: Filled missing values with mean 3.8830
Item 2: Filled missing values with mean 3.2345


In [25]:
user_item_matrix[target_items].isna().sum()

movieId
1    0
2    0
dtype: int64

#### 3- Calculate the average rating for each item 

In [26]:
all_item_means = user_item_matrix.mean(axis=0)

In [27]:
all_item_means.head()

movieId
1    3.883013
2    3.234476
3    3.045141
5    2.968506
6    3.882687
dtype: float64

In [28]:
all_item_means.shape

(1000,)

#### 4- For each item, calculate the difference between ratings and the mean rating of the item

In [42]:
centered_matrix = user_item_matrix - all_item_means
centered_matrix = centered_matrix.fillna(0)

In [43]:
centered_matrix

movieId,1,2,3,5,6,7,10,11,16,17,...,168252,171763,174055,176371,177765,187593,195159,202439,204698,207313
userId,,,,,,,,,,,,,,,,,,,,,
1,4.440892e-16,4.440892e-16,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.096055,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,4.440892e-16,2.655245e-01,0.0,0.0,0.000000,0.000000,0.594007,0.382421,0.0,1.096055,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
10,-1.383013e+00,-1.234476e+00,0.0,0.0,0.000000,0.000000,0.594007,0.000000,0.0,0.000000,...,0.116827,0.0,0.000000,-0.446614,0.0,0.0,0.0,0.0,0.0,0.0
13,4.440892e-16,4.440892e-16,0.0,0.0,1.117313,0.000000,0.000000,-0.617579,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
15,4.440892e-16,4.440892e-16,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.596055,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200943,-3.830129e-01,-7.344755e-01,0.0,0.0,-0.382687,1.190465,0.094007,0.000000,0.0,1.096055,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
200944,1.169871e-01,4.440892e-16,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
200945,4.440892e-16,4.440892e-16,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,-3.278539,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


#### 5- compute the covariance for each two items

$$\text{Cov}(i, j) = \frac{\sum (\text{centered rating}_i \times \text{centered rating}_j)}{n - 1}$$

In [44]:
covariance_matrix = centered_matrix.cov()

In [45]:
covariance_matrix

movieId,1,2,3,5,6,7,10,11,16,17,...,168252,171763,174055,176371,177765,187593,195159,202439,204698,207313
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.418449,0.049548,0.011785,0.013561,0.015609,0.011142,0.031787,0.017154,0.009247,0.016423,...,0.012015,0.008247,0.006514,0.006796,0.014630,0.006935,0.011349,0.006268,0.005624,0.008137
2,0.049548,0.224923,0.014982,0.018806,0.010168,0.013720,0.029938,0.016465,0.009124,0.009469,...,0.006582,0.004138,0.002838,0.002844,0.005192,0.005361,0.003592,0.001706,0.003852,0.003776
3,0.011785,0.014982,0.081131,0.017197,0.005044,0.011098,0.010915,0.010921,0.004250,0.004230,...,0.001180,0.001032,0.000354,0.000608,0.000823,0.000689,0.000477,0.000457,0.000711,0.000571
5,0.013561,0.018806,0.017197,0.083318,0.004984,0.013653,0.011842,0.013853,0.003839,0.007446,...,0.001085,0.001017,0.000706,0.000905,0.000976,0.000160,0.000374,-0.000040,0.000343,0.000331
6,0.015609,0.010168,0.005044,0.004984,0.170953,0.005660,0.018157,0.007273,0.031029,0.005935,...,0.004100,0.003987,0.004366,0.005297,0.002391,0.001740,0.002081,0.003031,0.003245,0.002747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187593,0.006935,0.005361,0.000689,0.000160,0.001740,-0.000095,0.002020,0.000652,0.001730,0.000946,...,0.016659,0.010108,0.006313,0.006896,0.008130,0.073367,0.012054,0.005269,0.010212,0.007961
195159,0.011349,0.003592,0.000477,0.000374,0.002081,0.000286,0.001933,0.000607,0.001359,0.001387,...,0.013824,0.009090,0.006148,0.007698,0.010599,0.012054,0.067787,0.007936,0.007429,0.009244
202439,0.006268,0.001706,0.000457,-0.000040,0.003031,0.000145,0.001045,0.000666,0.002396,0.000898,...,0.006282,0.006783,0.006869,0.008117,0.005514,0.005269,0.007936,0.055051,0.010112,0.008714


In [46]:
covariance_matrix.iloc[:5, :5]

movieId,1,2,3,5,6
movieId,,,,,
1,0.418449,0.049548,0.011785,0.013561,0.015609
2,0.049548,0.224923,0.014982,0.018806,0.010168
3,0.011785,0.014982,0.081131,0.017197,0.005044
5,0.013561,0.018806,0.017197,0.083318,0.004984
6,0.015609,0.010168,0.005044,0.004984,0.170953


In [47]:
covariance_matrix.shape

(1000, 1000)

In [48]:
item_A = covariance_matrix.columns[0]
item_B = covariance_matrix.columns[1]

covariance_matrix.loc[item_A, item_B]

np.float64(0.049548134063357306)

#### 6- Generate the covariance matrix

In [49]:
cov_matrix = centered_matrix.cov()

In [50]:
nan_count = cov_matrix.isna().sum().sum()
nan_count

np.int64(0)

#### 7- Determine the top 5-peers and top 10-peers for each of the target items (i1 and I2) using the transformed representation (covariance matrix)

In [ ]:
for item_id in target_items:
    if item_id in cov_matrix.index:
        item_covariances = cov_matrix[item_id]
        
        sorted_peers = item_covariances.sort_values(ascending=False)
        
        sorted_peers = sorted_peers.drop(item_id)
        
        top_5_peers = sorted_peers.head(5)
        top_10_peers = sorted_peers.head(10)
        for rank, (peer_id, score) in enumerate(top_5_peers.items(), 1):
            print(f"{rank}. Item {peer_id} (Covariance: {score:.4f})")

1. Item 3114 (Covariance: 0.1419)
2. Item 364 (Covariance: 0.1124)
3. Item 4886 (Covariance: 0.1122)
4. Item 480 (Covariance: 0.1093)
5. Item 6377 (Covariance: 0.1068)
1. Item 500 (Covariance: 0.0701)
2. Item 480 (Covariance: 0.0701)
3. Item 780 (Covariance: 0.0658)
4. Item 367 (Covariance: 0.0646)
5. Item 586 (Covariance: 0.0625)


#### 8- Determine reduced dimensional space for each user in case of using the top 5-peers

In [52]:
reduced_space_matrices = {}

for item_id in target_items:
    if item_id in cov_matrix.index:
        sorted_peers = cov_matrix[item_id].sort_values(ascending=False).drop(item_id)
        
        top_5_peer_ids = sorted_peers.head(5).index.tolist()
        
        reduced_matrix = centered_matrix[top_5_peer_ids]
        
        reduced_space_matrices[item_id] = reduced_matrix
        print(top_5_peer_ids)
        print(reduced_matrix.shape)

[3114, 364, 4886, 480, 6377]
(100000, 5)
[500, 480, 780, 367, 586]
(100000, 5)


#### 9- Use the results from point 8 compute the rating predictions od the original missing rating for each of the target items (I1 and I2) using the top 5-peers.

$$\text{Prediction}_{u, \text{target}} = \text{Mean}_{\text{target}} + \frac{\sum (\text{Covariance}_{\text{target, peer}} \times \text{CenteredRating}_{u, \text{peer}})}{\sum |\text{Covariance}_{\text{target, peer}}|}$$

In [53]:
for item_id in target_items:
    if item_id in reduced_space_matrices:
      
        reduced_matrix = reduced_space_matrices[item_id]
        
        peers_indices = reduced_matrix.columns
        weights = cov_matrix.loc[item_id, peers_indices]
        

        weighted_sum = reduced_matrix.dot(weights)
        
        sum_abs_weights = weights.abs().sum()
        
        target_mean = item_means[item_id]
        
        predicted_ratings = target_mean + (weighted_sum / sum_abs_weights)

        print(target_mean)
        print(sum_abs_weights)
        print(predicted_ratings.head(10))
        
    

3.8974375697494095
0.5826491190672849
userId
1     3.897438
3     4.025534
10    3.564512
13    3.897438
15    3.451021
16    3.130046
17    3.954850
18    4.477284
20    4.736934
25    3.805992
dtype: float64
3.2757576805978412
0.33308564459225387
userId
1     3.275758
3     3.281020
10    2.344610
13    3.275758
15    2.838070
16    3.275758
17    3.474556
18    3.275758
20    3.766166
25    3.234971
dtype: float64


#### 10- Determine reduced dimensional space for each user in case of using the top 10-peers.

In [54]:
reduced_spaces_10 = {}

for item_id in target_items:
    if item_id in cov_matrix.index:

        peers_indices = cov_matrix[item_id].nlargest(11).index.drop(item_id)
        

        reduced_matrix_10 = centered_matrix[peers_indices]
        
        reduced_spaces_10[item_id] = reduced_matrix_10
        

        print(peers_indices.tolist())
        print(reduced_matrix_10.shape)
        print(reduced_matrix_10.head())

[3114, 364, 4886, 480, 6377, 4306, 588, 1270, 260, 8961]
(100000, 10)
movieId  3114      364       4886      480       6377      4306      588   \
userId                                                                      
1         0.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3         0.0  0.184907  0.181836  0.306149  0.000000 -0.226194  0.294883   
10        0.0  0.184907  0.000000 -0.693851 -1.300425 -0.226194 -0.705117   
13        0.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
15        0.0  0.000000 -2.318164  0.000000  0.000000 -1.726194  0.000000   

movieId      1270      260   8961  
userId                             
1        1.030893  0.893134   0.0  
3        0.030893 -0.106866   0.0  
10      -0.469107 -0.106866   0.0  
13      -0.969107  0.000000   0.0  
15      -0.469107  0.000000   0.0  
[500, 480, 780, 367, 586, 364, 736, 1580, 356, 588]
(100000, 10)
movieId      500       480       780       367      586       364       7

#### 11- Use the results from point 10 to compute the rating predictions of the original missing rating for each of target items (I1 and I2) using the top 10-peers.

In [55]:
for item_id in target_items:
    if item_id in reduced_spaces_10:

        reduced_matrix = reduced_spaces_10[item_id]
        
 
        peers_indices = reduced_matrix.columns
        weights = cov_matrix.loc[item_id, peers_indices]
       
        weighted_sum = reduced_matrix.dot(weights)
        
        sum_abs_weights = weights.abs().sum()
        
        target_mean = item_means[item_id]
        
        predicted_ratings = target_mean + (weighted_sum / sum_abs_weights)
        

        print(target_mean)
        print(sum_abs_weights)
        print(predicted_ratings.head(10))

3.8974375697494095
1.0619289530357414
userId
1     4.063239
3     3.967686
10    3.573935
13    3.812386
15    3.441055
16    3.476392
17    4.004265
18    4.393105
20    4.676686
25    3.754911
dtype: float64
3.2757576805978412
0.6026270001778344
userId
1     3.101788
3     3.319098
10    2.704178
13    3.360265
15    3.033838
16    3.053678
17    3.423686
18    3.495628
20    3.414932
25    3.253214
dtype: float64


#### 12- Compare the results of point 9 with results of point 11. comment your answer.

In [56]:
# in point 9: Due to the small k, 
# the calculations here is most strict in the similarity between the items, 
# which cause to have higher variance but less bias if the top 5 peers are very similar to the target item,
# and since the highest covariance between them was 0.1419, 
# it's obvious to have higher variance in the predictions
# but in point 10:adding more k, cause to have more smoothing in the predictions,
# which can decrease the variance in the output.